In [1]:
!pip install datasets -q 

     |████████████████████████████████| 365 kB 5.3 MB/s 
     |████████████████████████████████| 115 kB 47.6 MB/s 
     |████████████████████████████████| 101 kB 9.0 MB/s 
     |████████████████████████████████| 212 kB 46.7 MB/s 
     |████████████████████████████████| 127 kB 40.8 MB/s 


In [15]:
!wget https://guillaumejaume.github.io/FUNSD/dataset.zip
!unzip dataset

--2022-08-20 14:02:49--  https://guillaumejaume.github.io/FUNSD/dataset.zip
Resolving guillaumejaume.github.io (guillaumejaume.github.io)... 185.199.109.153, 185.199.110.153, 185.199.111.153, ...
Connecting to guillaumejaume.github.io (guillaumejaume.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16838830 (16M) [application/zip]
Saving to: ‘dataset.zip’

dataset.zip         100%[===================>]  16.06M  --.-KB/s    in 0.1s    

2022-08-20 14:02:50 (136 MB/s) - ‘dataset.zip’ saved [16838830/16838830]



In [82]:
import json


def get_element_by_id(id,data=data):
  return next(item for item in data["form"] if item["id"] ==id)

def get_element_by_label(label,data=data):
  return next(item for item in data["form"] if item["label"] == label)

def get_all_ids_by_label(label,data=data):
  return [item["id"] for item in data["form"] if item["label"] == label]

def get_first_header(data=data):
  return next(item for item in data["form"] if item["label"] == "header")

def group_question_and_answers(question_id,data=data):
  question = get_element_by_id(question_id,data=data)
  if len(question["linking"]) > 0:
    answer = get_element_by_id(question["linking"][0][1],data=data)
    if question["text"] != answer["text"]:
      return {"question":question["text"],"answer":answer["text"]}



def load_json(path):
  with open(path, "r") as fi:
    data=json.loads(fi.read())
  return data

def create_donut_document(data):
  header = get_first_header(data)

  all_questions = get_all_ids_by_label("question",data=data)

  pairs=[]
  for q_id in all_questions:
    pair = group_question_and_answers(q_id,data=data)
    if pair is not None:
      pairs.append(pair)

  return {"header":header["text"], "q&a":pairs}

data = load_json("dataset/testing_data/annotations/89856243.json")

create_donut_document(data)

{'header': 'MARKETING RESEARCH RPROJECT APPROVAL',
 'q&a': [{'question': 'PRODUCT', 'answer': 'Triumph'},
  {'question': 'Date', 'answer': 'October 17, 1979'},
  {'question': 'Incidence', 'answer': '-'},
  {'question': 'Topline', 'answer': '2 wks from start of fid.'},
  {'question': 'Final', 'answer': '4 wks from start of fld'},
  {'question': 'PROJECT TITLE', 'answer': 'Triumph Disaster Check Study'},
  {'question': 'Key Criteria for Analysis',
   'answer': '-Triumph switchers asked why no longer smoking Triumph'},
  {'question': 'Research Limitations',
   'answer': 'Due to sample size and experience we feel 25 respondents will presently be Triumph smokers. An action standard of 75% should be utilized for judgments on product negatives only among this group'},
  {'question': 'Research Firm', 'answer': 'The Data Group. Inc'},
  {'question': 'Lenght Int', 'answer': '10 minutes'},
  {'question': '# Open ', 'answer': '4'},
  {'question': '# Reports', 'answer': '1'},
  {'question': 'Cost E

In [78]:
import os 
from pathlib import Path 

def create_donut_meta_data(path,donut_dir="dataset/testing_data/donut_annotations"):
  path = Path(path)
  donut_dir = Path(donut_dir)
  donut_dir.mkdir(parents=True, exist_ok=True)

  for file_name in path.iterdir():
    file_data=load_json(file_name)
    donut_doc=create_donut_document(file_data)
    with open(f"{donut_dir.joinpath(file_name.stem)}.json","w") as wr:
      json.dump(donut_doc,wr)
  
create_donut_meta_data("dataset/testing_data/annotations")


